<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Instrumenting a chatbot with human feedback</h1>

Phoenix provides endpoints to associate user-provided feedback directly with OpenInference spans as annotations.

In this tutorial, we will create a manually-instrument chatbot with user-triggered "👍" and "👎" feedback buttons. We will have those buttons trigger a callback that sends the user feedback to Phoenix and is viewable alongside the span. Automating associating feedback with spans is a powerful way to quickly focus on traces of your application that are not behaving as expected.

In [ ]:
!pip install arize-phoenix

In [1]:
import json
import os
from getpass import getpass
from typing import Any, Dict
from uuid import uuid4

import httpx
import ipywidgets as widgets
from IPython.display import display
from openinference.semconv.trace import (
    OpenInferenceMimeTypeValues,
    OpenInferenceSpanKindValues,
    SpanAttributes,
)
from opentelemetry import trace as trace_api

import phoenix as px
from phoenix.client import Client
from phoenix.otel import register

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

## Define endpoints and configure OpenTelemetry tracing

In [ ]:
px.launch_app()

In [ ]:
tracer_provider = register(endpoint="http://127.0.0.1:6006/v1/traces")

In [2]:
ENDPOINT = "http://localhost:6006/v1"
FEEDBACK_ENDPOINT = f"{ENDPOINT}/span_annotations"
OPENAI_API_URL = "https://api.openai.com/v1/chat/completions"
TRACER = trace_api.get_tracer(__name__)

## Define and instrument chat service backend

Here we define two functions:

`generate_response` is a function that contains the chatbot logic for responding to a user query. `generate_response` is manually instrumented using the `OpenInference` semantic conventions. More information on how to manually instrument an application can be found [here](https://docs.arize.com/phoenix/tracing/how-to-tracing/manual-instrumentation). `generate_response` also returns the OpenTelemetry spanID, a hex-encoded string that is used to associate feedback with a specific trace.

`send_feedback` is a function that sends user feedback to Phoenix via the `span_annotations` REST route.

In [4]:
client = Client()
http_client = httpx.Client()


def generate_response(
    input_text: str, model: str = "gpt-4o-mini", temperature: float = 0.1
) -> Dict[str, Any]:
    user_message = {"role": "user", "content": input_text, "uuid": str(uuid4())}
    invocation_parameters = {"temperature": temperature}
    payload = {
        "model": model,
        **invocation_parameters,
        "messages": [user_message],
    }
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai_api_key}",
    }
    with TRACER.start_as_current_span("chatbot with feedback example") as span:
        span.set_attribute(
            SpanAttributes.OPENINFERENCE_SPAN_KIND, OpenInferenceSpanKindValues.LLM.value
        )
        span.set_attribute(SpanAttributes.LLM_MODEL_NAME, payload["model"])
        span.set_attribute(SpanAttributes.INPUT_VALUE, json.dumps(payload["messages"][0]))
        span.set_attribute(SpanAttributes.INPUT_MIME_TYPE, OpenInferenceMimeTypeValues.JSON.value)

        # get the active hex-encoded spanID
        span_id = span.get_span_context().span_id.to_bytes(8, "big").hex()
        print(span_id)

        response = http_client.post(OPENAI_API_URL, headers=headers, json=payload)

        if not (200 <= response.status_code < 300):
            raise Exception(f"Failed to call OpenAI API: {response.text}")
        response_json = response.json()

        span.set_attribute(SpanAttributes.OUTPUT_VALUE, json.dumps(response_json))
        span.set_attribute(SpanAttributes.OUTPUT_MIME_TYPE, OpenInferenceMimeTypeValues.JSON.value)

    return response_json, span_id


def send_feedback(span_id: str, feedback: int) -> None:
    label = "👍" if feedback == 1 else "👎"
    client.annotations.add_span_annotation(
        span_id=span_id,
        annotation_name="user_feedback",
        label=label,
        score=feedback,
    )
    print(f"Feedback sent for span_id {span_id}: {label}")

## Create Chat Widget

We create a simple chat application using IPython widgets. Alongside the chatbot responses we provide feedback buttons that a user can click to provide feedback. These can be seen inside the Phoenix UI!

In [ ]:
def send_message(_):
    input_text = input_box.value

    # Send the message to the OpenAI API and get the response
    response_data, span_id = generate_response(input_text)
    assistant_content = response_data["choices"][0]["message"]["content"]

    # Create thumbs up and thumbs down buttons
    thumbs_up = widgets.Button(description="👍", layout=widgets.Layout(width="30px"))
    thumbs_down = widgets.Button(description="👎", layout=widgets.Layout(width="30px"))

    # Set up the callbacks for the buttons
    thumbs_up.on_click(lambda _: send_feedback(span_id, 1))
    thumbs_down.on_click(lambda _: send_feedback(span_id, 0))

    # Create a horizontal box to hold the response and the buttons
    response_box = widgets.HBox(
        [widgets.Label(f"Bot: {assistant_content}"), thumbs_up, thumbs_down]
    )

    # Add the user's message and the response to the chat history
    chat_history.children += (widgets.Label(f"You: {input_text}"), response_box)

    # Clear the input box
    input_box.value = ""


# Set up the chat interface
chat_history = widgets.VBox()
input_box = widgets.Text(placeholder="Type your message here...")
send_button = widgets.Button(description="Send")
send_button.on_click(send_message)

# Display the chat interface
display(chat_history, input_box, send_button)

## Analyze feedback using the Phoenix Client

We can use the Phoenix client to pull the annotated spans. By combining `get_spans_dataframe`
and `get_span_annotations_dataframe` we can create a dataframe of all annotations alongside
span data for analysis!

In [1]:
spans_df = client.spans.get_spans_dataframe(project_identifier="default")
annotations_df = client.spans.get_span_annotations_dataframe(
    spans_dataframe=spans_df, project_identifier="default"
)

In [ ]:
annotations_df.join(spans_df, how="inner", lsuffix="_annotation", rsuffix="_span")

In [ ]:
client.spans.get_span_annotations(
    span_ids=spans_df.index, project_identifier="default"
)